In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [3]:
#load data
dataset = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
print(dataset.shape)
print(test.shape)
dataset.head()

(43266, 9)
(18543, 8)


,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No,J'ai obtenu un compte de crédit de soins pour ...


In [3]:
# seperate X and y
complain_id = test["Complaint-ID"]
test = test.drop('Complaint-ID', axis=1)
X = dataset.copy().drop('Complaint-Status', axis=1)
X = X.drop('Complaint-ID', axis=1)
y = dataset['Complaint-Status']
print(X.shape)
print(y.shape)
print(X.columns)
print(test.columns)

(43266, 7)
(43266,)
Index(['Date-received', 'Transaction-Type', 'Complaint-reason',
       'Company-response', 'Date-sent-to-company', 'Consumer-disputes',
       'Consumer-complaint-summary'],
      dtype='object')
Index(['Date-received', 'Transaction-Type', 'Complaint-reason',
       'Company-response', 'Date-sent-to-company', 'Consumer-disputes',
       'Consumer-complaint-summary'],
      dtype='object')


In [4]:
df_merge = X.append(test)
print(df_merge.shape)
print(X.shape , test.shape)

(61809, 7)
(43266, 7) (18543, 7)


In [5]:
43266+18543

61809

In [6]:
for col in df_merge.columns:
    print(col,':', len(df_merge[col].unique()))

Date-received : 921
Transaction-Type : 18
Complaint-reason : 152
Company-response : 11
Date-sent-to-company : 931
Consumer-disputes : 3
Consumer-complaint-summary : 61362


In [7]:
#check missing values
df_merge.columns[df_merge.isnull().any()]

Index(['Company-response', 'Consumer-disputes'], dtype='object')

In [8]:
#missing value counts in each of these columns
miss = df_merge.isnull().sum()/len(df_merge)
miss = miss[miss > 0]
miss.sort_values(inplace=True)
miss

Consumer-disputes    0.178000
Company-response     0.521073
dtype: float64

In [9]:
df_merge['Consumer-complaint-summary'][:5]

0    Seterus, Inc a déposé un faux rapport auprès d...
1    XX / XX / XXXX La requête en faillite n ° XXXX...
2    El XXXX / XXXX / 15, estaba preparando el vuel...
3    The loan was paid in XXXX XXXX. In XXXX, 4 yea...
4    J'ai obtenu un compte de crédit de soins pour ...
Name: Consumer-complaint-summary, dtype: object

In [10]:
df_merge = df_merge.drop('Consumer-complaint-summary', axis=1)
df_merge.head()

,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes
0,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Yes
1,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,No
2,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,No
3,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,No
4,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,No


In [11]:
status = set(y)
count = 1
complain_stat = {}
for stat in status:
    if stat not in complain_stat:
        complain_stat[stat] = count
        count += 1
complain_stat

{'Closed with explanation': 1,
 'Closed': 2,
 'Untimely response': 3,
 'Closed with non-monetary relief': 4,
 'Closed with monetary relief': 5}

In [12]:
# dataset['Complaint-Status'] = list(dataset['Complaint-Status'].map(complain_stat))
df_merge.head()

,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes
0,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Yes
1,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,No
2,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,No
3,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,No
4,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,No


In [13]:
df_merge.dtypes

Date-received           object
Transaction-Type        object
Complaint-reason        object
Company-response        object
Date-sent-to-company    object
Consumer-disputes       object
dtype: object

In [14]:
df_merge.columns[df_merge.isnull().any()]

Index(['Company-response', 'Consumer-disputes'], dtype='object')

In [15]:
df_merge['Company-response'][pd.isnull(df_merge['Company-response'])]  = 'NaN'
df_merge['Consumer-disputes'][pd.isnull(df_merge['Consumer-disputes'])]  = 'NaN'

# dataset['Company-response'].fillna(0) # a[pd.isnull(a)]  = 'NaN'
# dataset['Consumer-disputes'].fillna(0) # a[pd.isnull(a)]  = 'NaN'
df_merge.head()

,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes
0,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Yes
1,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,No
2,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,No
3,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,No
4,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,No


In [16]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

for column in df_merge.columns:
    if df_merge[column].dtype == type(object):
        le = LabelEncoder()
        df_merge[column] = le.fit_transform(df_merge[column])
df_merge.head()

,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes
0,128,10,79,10,135,2
1,736,5,72,7,743,1
2,553,0,147,10,560,1
3,131,7,37,4,138,1
4,896,3,101,9,906,1


In [17]:
X_new = df_merge.iloc[:43266, :]
test_new = df_merge.iloc[43266:, :]

In [18]:
print(X.shape, X_new.shape)
print(test.shape, test_new.shape)

(43266, 7) (43266, 6)
(18543, 7) (18543, 6)


# **Making the predictions**

In [19]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.25, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

/home/aakash/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/aakash/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/aakash/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


In [24]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=0,
  shrinking=True, tol=0.001, verbose=False)

In [25]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred

array(['Closed with explanation', 'Closed with explanation',
       'Closed with explanation', ..., 'Closed with explanation',
       'Closed with explanation', 'Closed with explanation'], dtype=object)

In [29]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[   0,  198,    0,    0,    0],
       [   0, 8579,    0,    0,    0],
       [   0,  698,    0,    0,    0],
       [   0, 1261,    0,    0,    0],
       [   0,   81,    0,    0,    0]])

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7931034482758621

In [28]:
test_pred = classifier.predict(test_new)
print(len(test_pred))
test_pred

18543


array(['Closed with explanation', 'Closed with explanation',
       'Closed with explanation', ..., 'Closed with explanation',
       'Closed with explanation', 'Closed with explanation'], dtype=object)

In [41]:
len(complain_id)

18543

In [42]:
submission = pd.DataFrame(
    {'Complaint-ID': complain_id,
     'Complaint-Status': test_pred
    })

In [43]:
print(submission.shape)
submission.head()

(18543, 2)


,Complaint-ID,Complaint-Status
0,Te-1,Closed with explanation
1,Te-2,Closed with explanation
2,Te-3,Closed with explanation
3,Te-4,Closed with explanation
4,Te-5,Closed with explanation


In [44]:
submission.to_csv('sub.csv', encoding='utf-8', index=False)